In [154]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import requests
import os
import pyodbc
from datetime import datetime
from sqlalchemy import create_engine, exc
from dotenv import load_dotenv

load_dotenv()

True

In [155]:
etf = 'QQQ'
range = 700 #to go back to jan 23, window 10
window = 100 # or 100

In [156]:
def polygon(stock_ticker, lim, wndw):
    # Set up the GET request
    polygon_url = 'https://api.polygon.io/v1/indicators/sma'
    polygon_key = os.environ['polygon_api_key']
    
    # Parameters for the request
    params = {
        'timespan': 'day',
        'adjusted': 'true',
        'window': wndw,
        'series_type': 'close',
        'order': 'asc',
        'limit': lim,
        'apiKey': polygon_key
    }
    
    # Construct the full URL
    full_url = f'{polygon_url}/{stock_ticker}'
    
    # Request the data
    response = requests.get(full_url, params=params)
    polygon_data = response.json()
    results = polygon_data['results']['values']
    
    df_polygon = pd.DataFrame(results)
    #df_polygon.head(5)
    
    # Convert the Unix timestamp to datetime
    df_polygon['date'] = pd.to_datetime(df_polygon['timestamp'], unit='ms')
    
    # Format the datetime to the desired string format (YYYY-MM-DD)
    df_polygon['date'] = df_polygon['date'].dt.strftime('%Y-%m-%d')
    
    # Drop the unix time column
    df_polygon = df_polygon.drop(columns=['timestamp'])
    
    # Filter the DataFrame to include only dates from 01/01/2023
    df_polygon = df_polygon[df_polygon['date'] >= '2023-01-01']

    
    # Add Symbol
    df_polygon['Symbol'] = etf
    return df_polygon
    


In [157]:
# Example usage
df = polygon(etf, range, window)
# Remove even rows
#df = df.iloc[1::2]
#df.head(5)

 # Remove rows with duplicate date values
df = df.drop_duplicates(subset=['date'])
#df.jead(30)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 254 entries, 0 to 451
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   value   254 non-null    float64
 1   date    254 non-null    object 
 2   Symbol  254 non-null    object 
dtypes: float64(1), object(2)
memory usage: 7.9+ KB


In [158]:
# Define the connection string for Windows Authentication
server = 'MoemenLaptop'
database = 'InvestmentPortfolio'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establish the connection
try:
    conn = pyodbc.connect(connection_string)
    print("Connection successful!")
    cursor = conn.cursor()

    # Iterate over rows of the DataFrame
    for index, row in df.iterrows():
        # Example SQL insert command
        sql_command = f"INSERT INTO SMA_{window}_Day (value, [date], symbol) VALUES (?, ?, ?)"
        
        # Execute the command
        cursor.execute(sql_command, tuple(row))

    # Commit the transaction
    conn.commit()
    
    print("Records Inserted!")

except Exception as e:
    print(f"Error: {e}")
    
    
conn.close()

Connection successful!
Records Inserted!
